In [1]:
import pandas as pd
import numpy as np
from os.path import exists
from tqdm import tqdm
# from numba import jit
import re

In [2]:
tf_idf = pd.read_table("./../MMSR_WT22_Task1_Data/id_lyrics_tf-idf_mmsr.tsv", index_col='id')
tf_idf.head()

,abl,accept,across,act,addict,afraid,age,ago,ah,ahead,...,yea,yeah,year,yellow,yes,yesterday,yet,yo,young,youth
id,,,,,,,,,,,,,,,,,,,,,
9jbSytob9XRzwvB6,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.150511
Njp6JPM8vitbhVJU,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
h48f46ZsT9h0Z5Dm,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.149783,0.0,0.0,0.0,0.000000
ZmXVK43zlqdeq6z8,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
PV5EXN6AIVBqvsLO,0.0,0.0,0.0,0.0,0.0,0.327025,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000


In [3]:
# Other Feature vectors to use
word2vec = pd.read_table("./../MMSR_WT22_Task1_Data/id_lyrics_word2vec_mmsr.tsv", index_col='id')
bert = pd.read_table("./../MMSR_WT22_Task1_Data/id_bert_mmsr.tsv", index_col='id')

In [4]:
genres = pd.read_table("./../MMSR_WT22_Task1_Data/id_genres_mmsr.tsv", index_col="id")

In [5]:
info = pd.read_table("./../MMSR_WT22_Task1_Data/id_information_mmsr.tsv", index_col="id")

In [6]:
# New solution, with hardcoded cosine similarity and df

def get_cosine_similarity(arr_a: np.array, arr_b: np.array):
    def func(d1, d2, divisor):
        return np.divide(d1 @ d2, divisor, np.zeros_like(divisor), where=divisor > 0)

    norms_a = np.linalg.norm(arr_a, axis=-1)
    norms_b = np.linalg.norm(arr_b, axis=-1) # todo why doesn't norms[indicies_test] work here?

    r = np.zeros((len(arr_a), len(arr_b)))
    for index, sample in enumerate(tqdm(arr_b)):
        r[:, index] = func(arr_a, sample, norms_a * norms_b[index])

    return r

In [7]:
def cosine_similarity(d1, d2):
    divisor = np.linalg.norm(d1) * np.linalg.norm(d2)
    if divisor == 0:
        return 0
    return (d1 @ d2) / divisor

In [ ]:
# # Some calculations
# print(
#     cosine_similarity(tf_idf.iloc[0].values, tf_idf.iloc[2].values),
#     cosine_similarity(tf_idf.iloc[0].values, tf_idf.iloc[3].values)
# )

In [8]:
def get_innerProduct_similarity(arr_a: np.array, arr_b: np.array):
    r = np.zeros((len(arr_a), len(arr_b)))
    for index, sample in enumerate(tqdm(arr_b)):
        r[:, index] = arr_a @ sample 
    return r

In [9]:
def inner_product(d1, d2):
    return (d1 @ d2) 

In [10]:
# # I am not sure if it is correct
def jaccard_formulation(d1, d2):
    divisor = np.linalg.norm(d1) + np.linalg.norm(d2) - (d1 @ d2)
    if divisor == 0:
        return 0
    return (d1 @ d2) / divisor


In [11]:
# New solution, with hardcoded cosine similarity and df

def get_jaccard_similarity(arr_a: np.array, arr_b: np.array):
    def func(d1, d2, divisor):
        d1_d2_product = d1 @ d2
        return np.divide(d1_d2_product, divisor - d1_d2_product, np.zeros_like(divisor), where=divisor > 0)

    norms_a = np.linalg.norm(arr_a, axis=-1)
    norms_b = np.linalg.norm(arr_b, axis=-1) # todo why doesn't norms[indicies_test] work here?

    r = np.zeros((len(arr_a), len(arr_b)))
    for index, sample in enumerate(tqdm(arr_b)):
        r[:, index] = func(arr_a, sample, norms_a + norms_b[index])

    return r

In [12]:
def getSongIdByQuery(query):
    artist, track =query.split(',')
    id_ = info[(info['artist'] == artist) & (info['song'] == track)].index.values[0]
    return id_

In [13]:
def topIdsQueries(dfQueries, topNumber):
    top_idsongs = pd.DataFrame(index=dfQueries.columns.values, columns=range(topNumber))
    for query in tqdm(dfQueries.columns.values): # For each query done
        top = dfQueries[query].sort_values(ascending=False).drop(axis=0, index=[query]).head(topNumber) # Top n songs values
        top_idsongs.loc[query] = top.index.values
    return top_idsongs

## TOP 100 Ids using each song as a query with cosine similarity

### TF-IDF -- Cosine Similarity

In [14]:
result = get_cosine_similarity(tf_idf.to_numpy(), tf_idf.to_numpy())
print(result.shape)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 76115/76115 [37:24<00:00, 33.91it/s]

(76115, 76115)


In [15]:
df_test = pd.DataFrame(data=result.T ,index=tf_idf.index.values, columns=tf_idf.index)

In [18]:
topIdsQueries(df_test, 100).to_csv('./topIds/top_ids_cosine_tfidf_complete.csv')

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 76115/76115 [35:05<00:00, 36.16it/s]


### Word2vec -- Cosine Similarity

In [9]:
result2 = get_cosine_similarity(word2vec.to_numpy(), word2vec.to_numpy())
print(result2.shape)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 76115/76115 [13:56<00:00, 91.03it/s]

(76115, 76115)


In [10]:
df_test2 = pd.DataFrame(data=result2.T ,index=word2vec.index.values, columns=word2vec.index)

In [11]:
topIdsQueries(df_test2, 100).to_csv('./topIds/top_ids_cosine_word2vec_complete.csv')

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 76115/76115 [21:56<00:00, 57.81it/s]


### Bert -- Cosine Similarity

In [12]:
result3 = get_cosine_similarity(bert.to_numpy(), bert.to_numpy())
print(result3.shape)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 76115/76115 [28:33<00:00, 44.42it/s]

(76115, 76115)


In [13]:
df_test3 = pd.DataFrame(data=result3.T ,index=bert.index.values, columns=bert.index)

In [14]:
topIdsQueries(df_test3, 100).to_csv('./topIds/top_ids_cosine_bert_complete.csv')

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 76115/76115 [23:03<00:00, 55.03it/s]


#### Only compare with first 100 songs

In [19]:
def distanceToSongs(idSong, similarity_function, df, features_vector):
    if idSong in df.columns.values:
        print("Already in data")
    else:
        songs = features_vector.index.values
        distances = [similarity_function(features_vector.loc[idSong], features_vector.loc[song]) for index,song in enumerate(songs)]
        df[idSong]  = distances 

In [22]:
index_values = tf_idf.index
df_cosineDistance_tfidf = pd.DataFrame(index=index_values)

for id_ in tqdm(tf_idf.index.values[:100]):
    distanceToSongs(id_, cosine_similarity, df_cosineDistance_tfidf, tf_idf)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [22:28<00:00, 13.48s/it]


In [23]:
index_values = word2vec.index
df_cosineDistance_word2vec = pd.DataFrame(index=index_values)

for id_ in tqdm(word2vec.index.values[:100]):
    distanceToSongs(id_, cosine_similarity, df_cosineDistance_word2vec, word2vec)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [17:40<00:00, 10.60s/it]


In [24]:
index_values = bert.index
df_cosineDistance_bert = pd.DataFrame(index=index_values)

for id_ in tqdm(bert.index.values[:100]):
    distanceToSongs(id_, cosine_similarity, df_cosineDistance_bert, bert)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [21:05<00:00, 12.66s/it]


In [35]:
# df_test.head()

In [36]:
# df_cosineDistance_tfidf.head()

## TOP 100 Ids using each song as a query with jaccard similarity

### TF-IDF -- Jaccard Similarity

In [15]:
result = get_jaccard_similarity(tf_idf.to_numpy(), tf_idf.to_numpy())
print(result.shape)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 76115/76115 [39:14<00:00, 32.33it/s]

(76115, 76115)


In [14]:
# get_jaccard_similarity(tf_idf.to_numpy(), tf_idf[:3].to_numpy())

In [16]:
df_test = pd.DataFrame(data=result.T ,index=tf_idf.index.values, columns=tf_idf.index)

In [17]:
topIdsQueries(df_test, 100).to_csv('./topIds/top_ids_jaccard_tfidf_complete.csv')

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 76115/76115 [28:46<00:00, 44.08it/s]


### Word2Vec -- Jaccard Similarity

In [18]:
result2 = get_jaccard_similarity(word2vec.to_numpy(), word2vec.to_numpy())
print(result2.shape)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 76115/76115 [15:30<00:00, 81.83it/s]

(76115, 76115)


In [19]:
df_test2 = pd.DataFrame(data=result2.T ,index=word2vec.index.values, columns=word2vec.index)

In [20]:
topIdsQueries(df_test2, 100).to_csv('./topIds/top_ids_jaccard_word2vec_complete.csv')

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 76115/76115 [28:16<00:00, 44.88it/s]


### Bert -- Jaccard Similarity

In [21]:
result3 = get_jaccard_similarity(bert.to_numpy(), bert.to_numpy())
print(result3.shape)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 76115/76115 [29:01<00:00, 43.70it/s]

(76115, 76115)


In [22]:
df_test3 = pd.DataFrame(data=result3.T ,index=bert.index.values, columns=bert.index)

In [23]:
topIdsQueries(df_test3, 100).to_csv('./topIds/top_ids_jaccard_bert_complete.csv')

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 76115/76115 [34:07<00:00, 37.18it/s]


## TOP 100 Ids using each song as a query with jaccard similarity

### TF-IDF -- Inner Product Similarity

In [14]:
result = get_innerProduct_similarity(tf_idf.to_numpy(), tf_idf.to_numpy())
print(result.shape)
df_test = pd.DataFrame(data=result.T ,index=tf_idf.index.values, columns=tf_idf.index)
topIdsQueries(df_test, 100).to_csv('./topIds/top_ids_innerProduct_tfidf_complete.csv')

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 76115/76115 [36:12<00:00, 35.04it/s]


(76115, 76115)


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 76115/76115 [25:08<00:00, 50.47it/s]


### Word2Vec --  Inner Product Similarity

In [15]:
result2 = get_innerProduct_similarity(word2vec.to_numpy(), word2vec.to_numpy())
print(result2.shape)
df_test2 = pd.DataFrame(data=result2.T ,index=word2vec.index.values, columns=word2vec.index)
topIdsQueries(df_test2, 100).to_csv('./topIds/top_ids_innerProduct_word2vec_complete.csv')

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 76115/76115 [14:00<00:00, 90.55it/s]


(76115, 76115)


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 76115/76115 [34:16<00:00, 37.02it/s]


### Bert --  Inner Product Similarity

In [16]:
result3 = get_innerProduct_similarity(bert.to_numpy(), bert.to_numpy())
print(result3.shape)
df_test3 = pd.DataFrame(data=result3.T ,index=bert.index.values, columns=bert.index)
topIdsQueries(df_test3, 100).to_csv('./topIds/top_ids_innerProduct_bert_complete.csv')

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 76115/76115 [30:20<00:00, 41.82it/s]


(76115, 76115)


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 76115/76115 [28:01<00:00, 45.26it/s]
